In [29]:
!pip install keras-tuner

In [30]:
import keras_tuner
import keras

In [31]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [32]:
print(tf.__version__)
print(keras_tuner.__version__)

2.17.1
1.4.7


In [33]:
#loading inbuild dataset


fashion_mnist=keras.datasets.fashion_mnist

In [34]:
## loading the data into training and testing part


(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

# Print the dimensions of the dataset
print('Train: images = ', train_images.shape)
print('Test: images = ', test_images.shape)


Train: images =  (60000, 28, 28)
Test: images =  (10000, 28, 28)


In [35]:
# spliiting the datats into pixels

train_images = train_images / 255.0
test_images = test_images / 255.0


In [36]:
#reshaping the dimenshion of the dataset


train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [37]:
# creating Convolution model

def build_model(hp):

  # creating 1st convolution 2d layer
  model = keras.Sequential([keras.layers.Conv2D(
      filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
      kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
      activation='relu',
      input_shape=(28,28,1)),

  # cerate 2nd convolution 2d layer
        keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),

    # create flattern layer after creating colvolution 2d layer

    keras.layers.Flatten(),

    # create dense layer finally

    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),

    # finally create output later

    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model


In [38]:
# use random search to find out which model will be best in cnn

from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [46]:
import keras_tuner as kt
tuner_search=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=6,directory='output',project_name="Mnist Fashion")

Reloading Tuner from output/Mnist Fashion/tuner0.json


In [48]:
# to find best  hyper parameter


tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)              #epochs  3run 3 trials

In [52]:
model=tuner_search.get_best_models(num_models=1)[0]

In [53]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 26, 26, 48)          │             480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 24, 24, 32)          │          13,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 18432)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 48)                  │         884,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             490 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 899,610 (3.43 MB)

 Trainable params: 899,610 (3.43 MB)

 Non-trainable params: 0 (0.00 B)

In [55]:
# training the model after compliting 3 epochs to 10 epcochs

model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9462 - loss: 0.1446 - val_accuracy: 0.8938 - val_loss: 0.3395
Epoch 5/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9527 - loss: 0.1240 - val_accuracy: 0.8940 - val_loss: 0.3465
Epoch 6/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9589 - loss: 0.1095 - val_accuracy: 0.9007 - val_loss: 0.3498
Epoch 7/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9672 - loss: 0.0922 - val_accuracy: 0.8980 - val_loss: 0.3690
Epoch 8/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9706 - loss: 0.0801 - val_accuracy: 0.8935 - val_loss: 0.4447
Epoch 9/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9749 - loss: 0.0695 - val_accuracy: 0.8980 - val_loss: 0.4469
Epoch 10/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9782 - loss: 0.0591 - val_accuracy: 0.8922 - val_loss: 0.5115
